In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
True


In [ ]:
from huggingface_hub import login

login()


Tokenization

In [ ]:
import pandas as pd
import re
import html
import warnings
import os
import torch
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# -------------------------------
# Suppress warnings
# -------------------------------
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# -------------------------------
# Paths
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/NewsSumm Dataset.xlsx"
SAVE_DIR  = "/content/drive/MyDrive/tokenized_gemma_batches"

TRAIN_DIR = os.path.join(SAVE_DIR, "train")
TEST_DIR  = os.path.join(SAVE_DIR, "test")

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_excel(DATA_PATH)
df = df.dropna(subset=["human_summary"])
print("Rows after cleaning:", len(df))

# -------------------------------
# Resample to 15,000 rows
# -------------------------------
df = df.sample(n=15000, random_state=42)
print("Rows after resampling:", len(df))

# -------------------------------
# Clean text
# -------------------------------
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text(" ")
    text = html.unescape(text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "", text)
    text = re.sub(r"\+?\d[\d\s\-]{8,}\d", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["human_summary"] = df["human_summary"].apply(clean_text)

# -------------------------------
# Train–test split (80/20)
# -------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Train rows:", len(train_df))
print("Test rows:", len(test_df))

train_texts = train_df["human_summary"].astype(str).tolist()
test_texts  = test_df["human_summary"].astype(str).tolist()

# -------------------------------
# Load Gemma tokenizer
# -------------------------------
MODEL_NAME = "google/gemma-2b"   # or: google/gemma-7b

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Gemma has no pad token → use EOS token
tokenizer.pad_token = tokenizer.eos_token

# -------------------------------
# Tokenization settings
# -------------------------------
BATCH_SIZE = 4
MAX_LEN = 128

def tokenize_and_save(texts, out_dir, split_name):
    print(f"\nTokenizing {split_name} set...")

    batch_id = 0

    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i+BATCH_SIZE]

        encoded = tokenizer(
            batch_texts,
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        batch_data = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids.clone()
        }

        save_path = os.path.join(out_dir, f"batch_{batch_id}.pt")
        torch.save(batch_data, save_path)

        if batch_id % 50 == 0:
            print(f"{split_name}: saved batch {batch_id}")

        batch_id += 1

    print(f"{split_name} tokenization done. Total batches: {batch_id}")

# -------------------------------
# Run tokenization
# -------------------------------
tokenize_and_save(train_texts, TRAIN_DIR, "TRAIN")
tokenize_and_save(test_texts, TEST_DIR, "TEST")

print("\nGemma summary-only batchwise tokenization completed ")


In [ ]:
!pip uninstall -y torch torchvision torchaudio bitsandbytes
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U bitsandbytes transformers peft accelerate rouge-score


model training

In [ ]:
import os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from bitsandbytes.optim import AdamW8bit

MODEL_NAME = "google/gemma-2b"
TRAIN_DIR = "/content/drive/MyDrive/tokenized_gemma_batches/train/"
SAVE_DIR = "/content/drive/MyDrive/gemma2b_qlora"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0}
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.train()

optim = AdamW8bit(model.parameters(), lr=2e-4)

files = sorted([f for f in os.listdir(TRAIN_DIR) if f.endswith(".pt")])

for i, fname in enumerate(files, 1):
    batch = torch.load(os.path.join(TRAIN_DIR, fname))

    out = model(
        input_ids=batch["input_ids"].cuda(),
        attention_mask=batch["attention_mask"].cuda(),
        labels=batch["labels"].cuda()
    )

    out.loss.backward()
    optim.step()
    optim.zero_grad()

    if i % 50 == 0:
        print("batch", i, "loss", out.loss.item())

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("QLoRA training finished ")


model evaluation

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from rouge_score import rouge_scorer

# --------------------
# Paths
# --------------------
TEST_DIR = "/content/drive/MyDrive/tokenized_gemma_batches/test/"
BASE_MODEL = "google/gemma-2b"
LORA_DIR = "/content/drive/MyDrive/gemma2b_qlora"

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------
# Load tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# --------------------
# Load base model in 4-bit
# --------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": 0}
)

# --------------------
# Load LoRA adapter
# --------------------
model = PeftModel.from_pretrained(base_model, LORA_DIR)
model.eval()

# --------------------
# ROUGE scorer
# --------------------
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

r1 = r2 = rl = 0.0
r11 = r21 = rl1 = 0.0
count = 0

files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(".pt")])

# --------------------
# Evaluation loop
# --------------------
for fname in files:

    batch = torch.load(os.path.join(TEST_DIR, fname))

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"]

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50
        )

    gen_only = outputs[:, input_ids.shape[1]:]
    preds = tokenizer.batch_decode(gen_only, skip_special_tokens=True)
    refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

    for pred, ref in zip(preds, refs):
        scores = scorer.score(ref, pred)

        r1 += scores["rouge1"].fmeasure
        r2 += scores["rouge2"].fmeasure
        rl += scores["rougeL"].fmeasure

        r11 += scores["rouge1"].precision
        r21 += scores["rouge2"].precision
        rl1 += scores["rougeL"].precision

        count += 1

# --------------------
# Final results
# --------------------
print("\nROUGE RESULTS")
print("ROUGE-1-f1_score:", round(r1 / count, 4))
print("ROUGE-2-f1_score:", round(r2 / count, 4))
print("ROUGE-L-f1_score:", round(rl / count, 4))
print("ROUGE-1-precision:", round(r11 / count, 4))
print("ROUGE-2-precision:", round(r21 / count, 4))
print("ROUGE-L-precision:", round(rl1 / count, 4))
